In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**This is a test for running Stable Diffusion (adapted NovelAI model), created by physics1024 2022.10.11 10:09**

In [3]:
!nvidia-smi

Tue Oct 11 13:16:07 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

test for GPUs

In [1]:
!mkdir /kaggle/working/content
%cd /kaggle/working/content

!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui
%cd stable-diffusion-webui

/kaggle/working/content
Cloning into 'stable-diffusion-webui'...
remote: Enumerating objects: 5519, done.
remote: Counting objects: 100% (122/122), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 5519 (delta 64), reused 99 (delta 53), pack-reused 5397
Receiving objects: 100% (5519/5519), 21.02 MiB | 8.58 MiB/s, done.
Resolving deltas: 100% (3849/3849), done.
/kaggle/working/content/stable-diffusion-webui


get stable difussion

In [ ]:
!mkdir -p /kaggle/working/content/stable-diffusion-webui/models/Stable-diffusion /kaggle/working/content/stable-diffusion-webui/models/hypernetworks
%cd /kaggle/working/content/stable-diffusion-webui/models/Stable-diffusion/


# Models

# 7G animefull-final-latest (optional)
!apt install -y -qq zstd
!apt autoremove
!curl -Lo /kaggle/working/content/stable-diffusion-webui/config.yaml https://cloudflare-ipfs.com/ipfs/bafybeiav3j7npiuewbel3mi32l3sidgkw54kuleosbhxmdvddbnvtfi7yu/config.yaml
!curl -L https://cloudflare-ipfs.com/ipfs/bafybeibhlcqjgvjtc3ck2bcazae5yizaetm7y7nm5rux674gsxgd4l5uce/animefull-latest.tar.zst | zstdcat | tar xf - -C /kaggle/working/content/stable-diffusion-webui/models/Stable-diffusion
!mv /kaggle/working/content/stable-diffusion-webui/models/Stable-diffusion/animefull-latest/model.ckpt /kaggle/working/content/stable-diffusion-webui/models/Stable-diffusion/animefull-final-latest.ckpt

# 4G animefull-final-pruned 
# !curl -Lo animefull-final-pruned.ckpt https://cloudflare-ipfs.com/ipfs/bafybeicpamreyp2bsocyk3hpxr7ixb2g2rnrequub3j2ahrkdxbvfbvjc4/model.ckpt
# !curl -Lo ~/content/stable-diffusion-webui/config.yaml https://cloudflare-ipfs.com/ipfs/bafybeiav3j7npiuewbel3mi32l3sidgkw54kuleosbhxmdvddbnvtfi7yu/config.yaml



# Others

# Install VAE Weights (optional)
!curl -Lo animefull-final-pruned.vae.pt https://cloudflare-ipfs.com/ipfs/bafybeiccldswdd3wvg57jhclcq53lvsc6gizasiblwayvhlv6eq4wow7wu/animevae.pt 
!ln -s animefull-final-pruned.vae.pt animefull-final-latest.vae.pt

# Install hypernetwork （optional)
!curl -L https://cloudflare-ipfs.com/ipfs/bafybeiduanx2b3mcvxlwr66igcwnpfmk3nc3qgxlpwh6oq6m6pxii3f77e/_modules.tar | tar x -C /kaggle/working/content/stable-diffusion-webui/models/hypernetworks

# Install embeddings (optional)
!curl -L https://cloudflare-ipfs.com/ipfs/bafybeie3hdjchxs5tz4n75bos53nhcklslguxchdurc2ynrzcfv2kwyklu/embeddings.tar | tar x -C /kaggle/working/content/stable-diffusion-webui/embeddi

In [3]:
!ls /kaggle/temp
!df -h

ls: cannot access '/kaggle/temp': No such file or directory
Filesystem        Size  Used Avail Use% Mounted on
overlay           4.0T  3.8T  147G  97% /
tmpfs              64M     0   64M   0% /dev
shm               7.0G     0  7.0G   0% /dev/shm
/dev/loop1         20G   56K   20G   1% /kaggle/lib
/dev/mapper/snap  4.0T  3.8T  147G  97% /home/jupyter
tmpfs             8.8G     0  8.8G   0% /proc/acpi
tmpfs             8.8G     0  8.8G   0% /proc/scsi
tmpfs             8.8G     0  8.8G   0% /sys/firmware


get model file

In [ ]:
%cd /kaggle/working/content/stable-diffusion-webui

# hacks to load model in colab
!sed -i 's/map_location="cpu"/map_location="cuda"/g' /kaggle/working/content/stable-diffusion-webui/modules/sd_models.py

# Use novelai stable-diffusion-private (maybe slow, optional)
# !mkdir -p repositories/stable-diffusion
# !curl -L https://cloudflare-ipfs.com/ipfs/bafybeiauciguui6skzcwg23mjprjbuqhfzudms2gfi4rna55bygc3s6yxa/stable-diffusion-private.tar | tar x -C repositories/stable-diffusion
# !git config --global --add safe.directory /kaggle/working/content/stable-diffusion-webui/repositories/stable-diffusion
# !pip3 install icecream xformers
# !sed -i 's/transformer\.text_model/transformer/g' modules/sd_hijack.py
# !COMMANDLINE_ARGS="--share --gradio-debug --config config.yaml --xformers" \
#  REQS_FILE="requirements.txt" STABLE_DIFFUSION_COMMIT_HASH="5d95f1742ab1831d99df91c1a469b2b56a61eebf" python launch.py

# ...Or use normal stable-diffusion
!COMMANDLINE_ARGS="--share --gradio-debug --config config.yaml --disable-safe-unpickle" REQS_FILE="requirements.txt" python launch.py --disable-safe-unpickle

test to adapt form colab

In [ ]:
!zip -q -r output.zip /kaggle/working/content/stable-diffusion-webui/outputs 


pack image 